In [153]:
import numpy as np

In [154]:
def polynom_matrix(a_min,a_max,b_min,b_max,seq_length): # USE THIS FOR COMPARISON TO FASTER FUNCTION
    a_range = a_max+1 - a_min
    b_range = b_max+1 - b_min
    
    Poly_matrix = np.fromfunction(lambda a,b,u,v: (a+a_min)*u*u + (b+b_min)*u*v, (a_range,b_range,seq_length,seq_length))
    
    return Poly_matrix

In [155]:
polynom_matrix(0,20,0,20,21)[1,1,2,3] # Should be 1*2*2 + 1*2*3 = 10

10.0

In [146]:
%timeit polynom_matrix(-10,10,-10,10,21)

5.44 ms ± 187 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [223]:
polynom_matrix(-1,1,-1,1,21)[2,2,1]

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21.])

In [174]:
polynom_matrix(0,20,0,20,21)[1,1,2,3]

10.0

Efficiently making a big matrix with entries
$$
au^2 + buv
$$

In [217]:
def make_P_array(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    a_range = np.arange(a_min, a_max+1)
    b_range = np.arange(b_min, b_max+1)
    u_range = np.arange(seq_length)
    v_range = u_range
    
    au_squared_matrix = np.multiply.outer(a_range, u_range*u_range) # a_len x seq_length
    au_tensor = np.broadcast_to(au_squared_matrix,(seq_length,b_len,a_len,seq_length))
    # seq_length x b_len x a_len x seq_length
    #k,b,a,k --> a,b,k,k
    
    bu_matrix = np.multiply.outer(b_range,u_range) # b_len x seq_length
    buv_matrix = np.multiply.outer(bu_matrix,v_range) # b_len x seq_len x seq_len
    buv_tensor = np.broadcast_to(buv_matrix,(a_len,b_len,seq_length,seq_length))
    
    corrected_au_tensor = np.swapaxes(au_tensor, 0,2)
    
    tensor = corrected_au_tensor + buv_tensor
    
    P_array = np.swapaxes(tensor, 2,3)
    
    return P_array

In [232]:
%timeit make_P_array(-20,20,-20,20,21) # *racecar noises*

1.42 ms ± 20.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [233]:
%timeit polynom_matrix(-20,20,-20,20,21)

23.3 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [230]:
make_P_array(0,10,0,10,21)[1,1,1]

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21])

In [231]:
polynom_matrix(0,10,0,10,21)[1,1,1]

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21.])